In [9]:
import sys
import os
import pandas as pd
from datetime import datetime
from typing import List, Dict, Optional
from sqlalchemy import create_engine, text
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

print("📦 라이브러리 임포트 완료")

# =============================================================================
# 2. 설정 및 연결
# =============================================================================

# DB 설정
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from config.settings import DatabaseConfig

db_config = DatabaseConfig()
DATABASE_URL = db_config.DATABASE_URL
engine = create_engine(DATABASE_URL, pool_pre_ping=True)

# LLM 클라이언트 설정
llm_client = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print(f"LLM Client initialized: {llm_client.model_name}")

print("⚙️ 설정 완료")

# =============================================================================
# 3. 데이터베이스 연결 테스트
# =============================================================================

def test_db_connection():
    """데이터베이스 연결 테스트"""
    try:
        with engine.connect() as connection:
            result = connection.execute(text("SELECT 1 as test"))
            print("✅ 데이터베이스 연결 성공")
            return True
    except Exception as e:
        print(f"❌ 데이터베이스 연결 실패: {e}")
        return False

# 연결 테스트
test_db_connection()


📦 라이브러리 임포트 완료
LLM Client initialized: gpt-4o-mini
⚙️ 설정 완료
✅ 데이터베이스 연결 성공


True

In [11]:
# =============================================================================
# 4. 요약 대상 팀 조회
# =============================================================================

def get_teams_to_summarize():
    """요약이 필요한 팀 목록 조회"""
    
    query = text("""
    SELECT 
        ef.team_evaluation_id,
        ef.period_id,
        t.team_name,
        COUNT(ef.evaluation_feedback_id) as feedback_count
    FROM evaluation_feedbacks ef
    JOIN team_evaluations te ON ef.team_evaluation_id = te.team_evaluation_id
    JOIN teams t ON te.team_id = t.team_id
    WHERE ef.content IS NOT NULL 
    AND ef.content != ''
    AND NOT EXISTS (
        SELECT 1 FROM evaluation_feedback_summaries efs 
        WHERE efs.team_evaluation_id = ef.team_evaluation_id 
        AND efs.period_id = ef.period_id
    )
    GROUP BY ef.team_evaluation_id, ef.period_id, t.team_name
    ORDER BY t.team_name
    """)
    
    try:
        with engine.connect() as connection:
            df = pd.read_sql(query, connection)
            print(f"📊 요약 대상 팀: {len(df)}개")
            return df.to_dict('records')
    except Exception as e:
        print(f"❌ 팀 목록 조회 실패: {e}")
        return []

# 요약 대상 팀 확인
teams_to_process = get_teams_to_summarize()
print("\n📋 처리 대상 팀:")
for i, team in enumerate(teams_to_process, 1):
    print(f"{i}. {team['team_name']} (피드백 {team['feedback_count']}개)")

# =============================================================================
# 5. 특정 팀의 피드백 조회
# =============================================================================

def get_team_feedbacks(team_evaluation_id, period_id):
    """특정 팀의 피드백 내용 조회"""
    
    query = text("""
    SELECT 
        ef.content
    FROM evaluation_feedbacks ef
    WHERE ef.team_evaluation_id = :team_evaluation_id 
    AND ef.period_id = :period_id
    AND ef.content IS NOT NULL 
    AND ef.content != ''
    ORDER BY ef.evaluation_feedback_id
    """)
    
    try:
        with engine.connect() as connection:
            result = connection.execute(query, {
                'team_evaluation_id': team_evaluation_id,
                'period_id': period_id
            })
            feedbacks = [{'content': row.content, 'emp_name': '팀원'} for row in result]
            return feedbacks
    except Exception as e:
        print(f"❌ 피드백 조회 실패: {e}")
        return []

# 샘플 피드백 확인 (첫 번째 팀)
if teams_to_process:
    first_team = teams_to_process[0]
    sample_feedbacks = get_team_feedbacks(
        first_team['team_evaluation_id'], 
        first_team['period_id']
    )
    
    print(f"\n📝 {first_team['team_name']} 팀 피드백 샘플:")
    for feedback in sample_feedbacks[:2]:  # 처음 2개만 표시
        print(f"- {feedback['emp_name']}: {feedback['content'][:50]}...")


📊 요약 대상 팀: 6개

📋 처리 대상 팀:
1. AI개발팀 (피드백 4개)
2. 기업영업팀 (피드백 1개)
3. 데이터분석팀 (피드백 3개)
4. 백엔드개발팀 (피드백 2개)
5. 퍼포먼스마케팅팀 (피드백 1개)
6. 프론트엔드팀 (피드백 2개)

📝 AI개발팀 팀 피드백 샘플:
- 팀원: 테스트 피드백...
- 팀원: 올해 성과평가에 대해 솔직한 의견을 전달드리고 싶습니다. 먼저 평가 기준이 명확하지 않다고...


In [12]:
# =============================================================================
# 6. LLM 요약 함수
# =============================================================================

def summarize_team_feedbacks(team_name, feedbacks):
    """팀 피드백을 LLM으로 요약"""
    
    # 피드백 텍스트 생성 (완전 익명)
    feedbacks_text = ""
    for i, feedback in enumerate(feedbacks, 1):
        feedbacks_text += f"{i}. {feedback['content']}\n"
    
    # 프롬프트 생성
    prompt = f"""
당신은 HR 전문가입니다. 다음은 {team_name} 팀 사원들이 팀장에게 익명으로 전달한 피드백입니다.
이 내용들을 팀장이 보기 쉽도록 건설적이고 부드러운 톤으로 요약해주세요.

## 익명 피드백 내용:
{feedbacks_text}

## 요약 지침:
1. 주요 피드백을 카테고리별로 분류해주세요
2. 공통적으로 언급되는 사항은 **굵게** 표시하여 강조해주세요
3. 개인을 특정하지 않고 전체적인 관점에서 요약해주세요
4. 팀장이 기분 나쁘지 않도록 부드럽고 건설적인 톤으로 작성해주세요
5. 비판보다는 개선 기회로 접근하여 긍정적으로 표현해주세요
6. 문장은 "-습니다", "-있습니다" 형태의 정중한 존댓말로 작성해주세요
7. 각 섹션 사이에 줄바꿈을 추가해서 읽기 쉽게 해주세요
8. 제목 없이 바로 내용부터 시작해주세요
9. 섹션 제목은 ## (큰제목)으로 표시해주세요
10. "불만", "문제" 같은 부정적인 단어 대신 "개선 희망사항", "발전 방향" 같은 긍정적인 표현을 사용해주세요

## 요약 형식:
## 1. [카테고리명]
- 팀원들이 개선을 희망하는 사항이 있습니다.
- **여러 팀원들이 공통적으로 언급한 발전 방향입니다.**
- 추가로 개선 기회가 있는 영역입니다.

## 2. [카테고리명]  
- 팀원들이 더 나은 환경을 위해 제안한 사항입니다.
- **반복적으로 언급된 중요한 개선 포인트입니다.**
- 성장을 위한 추가 제안사항입니다.

## 개선 방향 제안
- 팀 발전을 위해 고려해볼 수 있는 방향입니다.
- **우선적으로 검토해볼 만한 개선 기회입니다.**
- 팀원들의 성장과 만족도 향상을 위한 제안입니다.

요약해주세요:
"""
    
    try:
        response = llm_client.invoke(prompt)
        summary = response.content.strip()
        return summary
        
    except Exception as e:
        print(f"❌ LLM 요약 실패: {e}")
        return None

# 샘플 요약 테스트 (첫 번째 팀)
if teams_to_process and sample_feedbacks:
    print(f"\n🤖 {first_team['team_name']} 팀 요약 생성 중...")
    sample_summary = summarize_team_feedbacks(first_team['team_name'], sample_feedbacks)
    
    if sample_summary:
        print("✅ 요약 생성 완료")
        print(f"\n📄 요약 결과 (미리보기):\n{sample_summary[:200]}...")
    else:
        print("❌ 요약 생성 실패")

# =============================================================================
# 7. 요약 결과 저장
# =============================================================================

def save_summary(team_evaluation_id, period_id, summary_content):
    """요약 결과를 데이터베이스에 저장"""
    
    query = text("""
    INSERT INTO evaluation_feedback_summaries 
    (team_evaluation_id, period_id, content)
    VALUES (:team_evaluation_id, :period_id, :content)
    ON DUPLICATE KEY UPDATE
    content = VALUES(content)
    """)
    
    try:
        with engine.connect() as connection:
            connection.execute(query, {
                'team_evaluation_id': team_evaluation_id,
                'period_id': period_id,
                'content': summary_content
            })
            connection.commit()
            return True
    except Exception as e:
        print(f"❌ 요약 저장 실패: {e}")
        return False



🤖 AI개발팀 팀 요약 생성 중...
✅ 요약 생성 완료

📄 요약 결과 (미리보기):
## 1. 성과 평가 기준
- 팀원들이 성과 평가 기준에 대한 명확성을 개선하기를 희망하고 있습니다.
- **여러 팀원들이 공통적으로 평가 기준의 명확성이 부족하다고 언급하였습니다.**
- 평가 기준에 대한 사전 설명과 구체적인 지침이 필요하다는 의견이 있습니다.

## 2. 피드백 과정
- 팀원들이 피드백 과정의 구체성을 높이기 위한 제안을 하고 있습니다...


In [13]:
# =============================================================================
# 8. 메인 처리 함수
# =============================================================================

def process_all_teams():
    """모든 팀의 피드백을 요약 처리"""
    
    print("🚀 팀별 피드백 요약 처리 시작")
    print("=" * 50)
    
    # 처리할 팀 목록 조회
    teams = get_teams_to_summarize()
    
    if not teams:
        print("📭 처리할 팀이 없습니다.")
        return
    
    success_count = 0
    total_count = len(teams)
    
    # 각 팀별로 처리
    for i, team in enumerate(teams, 1):
        team_name = team['team_name']
        team_evaluation_id = team['team_evaluation_id']
        period_id = team['period_id']
        
        print(f"\n📋 [{i}/{total_count}] {team_name} 처리 중...")
        
        try:
            # 1. 피드백 조회
            feedbacks = get_team_feedbacks(team_evaluation_id, period_id)
            
            if not feedbacks:
                print(f"⚠️  {team_name}: 피드백 없음, 건너뜀")
                continue
            
            # 2. LLM 요약 생성
            print(f"🤖 {team_name}: 요약 생성 중... (피드백 {len(feedbacks)}개)")
            summary = summarize_team_feedbacks(team_name, feedbacks)
            
            if not summary:
                print(f"❌ {team_name}: 요약 생성 실패")
                continue
            
            # 3. 요약 저장
            if save_summary(team_evaluation_id, period_id, summary):
                print(f"✅ {team_name}: 완료")
                success_count += 1
            else:
                print(f"❌ {team_name}: 저장 실패")
                
        except Exception as e:
            print(f"❌ {team_name}: 처리 중 오류 - {e}")
            continue
    
    # 최종 결과
    print("\n" + "=" * 50)
    print(f"🎯 처리 완료: {success_count}/{total_count} 성공")
    print(f"📊 성공률: {success_count/total_count*100:.1f}%" if total_count > 0 else "")

# =============================================================================
# 9. 특정 팀만 처리하는 함수
# =============================================================================

def process_specific_team(team_evaluation_id, period_id):
    """특정 팀의 피드백만 처리"""
    
    print(f"🎯 특정 팀 처리 시작 - Team Evaluation ID: {team_evaluation_id}, Period ID: {period_id}")
    
    try:
        # 1. 해당 팀 정보 조회
        team_query = text("""
        SELECT t.team_name
        FROM team_evaluations te
        JOIN teams t ON te.team_id = t.team_id
        WHERE te.team_evaluation_id = :team_evaluation_id
        """)
        
        with engine.connect() as connection:
            result = connection.execute(team_query, {'team_evaluation_id': team_evaluation_id})
            team_row = result.fetchone()
            
            if not team_row:
                print("❌ 해당 팀을 찾을 수 없습니다.")
                return False
            
            team_name = team_row._mapping['team_name']
        
        # 2. 피드백 조회
        feedbacks = get_team_feedbacks(team_evaluation_id, period_id)
        
        if not feedbacks:
            print(f"⚠️ {team_name}: 피드백 없음")
            return False
        
        # 3. LLM 요약 생성
        print(f"🤖 {team_name}: 요약 생성 중... (피드백 {len(feedbacks)}개)")
        summary = summarize_team_feedbacks(team_name, feedbacks)
        
        if not summary:
            print(f"❌ {team_name}: 요약 생성 실패")
            return False
        
        # 4. 요약 저장
        if save_summary(team_evaluation_id, period_id, summary):
            print(f"✅ {team_name}: 완료")
            return True
        else:
            print(f"❌ {team_name}: 저장 실패")
            return False
            
    except Exception as e:
        print(f"❌ 특정 팀 처리 중 오류: {e}")
        return False

# =============================================================================
# 10. 결과 확인 함수
# =============================================================================

def check_summary_results():
    """저장된 요약 결과 확인"""
    
    query = text("""
    SELECT 
        t.team_name,
        p.period_name,
        LENGTH(efs.content) as summary_length,
        LEFT(efs.content, 100) as summary_preview
    FROM evaluation_feedback_summaries efs
    JOIN team_evaluations te ON efs.team_evaluation_id = te.team_evaluation_id
    JOIN teams t ON te.team_id = t.team_id
    LEFT JOIN periods p ON efs.period_id = p.period_id
    ORDER BY t.team_name
    """)
    
    try:
        with engine.connect() as connection:
            df = pd.read_sql(query, connection)
            print("📊 저장된 요약 결과:")
            print(df.to_string(index=False))
    except Exception as e:
        print(f"❌ 결과 확인 실패: {e}")

def get_detailed_summary(team_evaluation_id, period_id):
    """특정 팀의 상세 요약 내용 확인"""
    
    query = text("""
    SELECT 
        t.team_name,
        p.period_name,
        efs.content
    FROM evaluation_feedback_summaries efs
    JOIN team_evaluations te ON efs.team_evaluation_id = te.team_evaluation_id
    JOIN teams t ON te.team_id = t.team_id
    LEFT JOIN periods p ON efs.period_id = p.period_id
    WHERE efs.team_evaluation_id = :team_evaluation_id 
    AND efs.period_id = :period_id
    """)
    
    try:
        with engine.connect() as connection:
            result = connection.execute(query, {
                'team_evaluation_id': team_evaluation_id,
                'period_id': period_id
            })
            row = result.fetchone()
            
            if row:
                print(f"📋 {row._mapping['team_name']} - {row._mapping['period_name']} 요약:")
                print("-" * 50)
                print(row._mapping['content'])
            else:
                print("해당 팀의 요약을 찾을 수 없습니다.")
                
    except Exception as e:
        print(f"❌ 상세 요약 조회 실패: {e}")


In [14]:
# =============================================================================
# 11. 실행 부분
# =============================================================================

print("\n🎯 팀별 피드백 요약 에이전트 준비 완료!")
print("\n실행 옵션:")
print("1. process_all_teams() - 모든 팀 처리")
print("2. process_specific_team(team_evaluation_id, period_id) - 특정 팀 처리")
print("3. check_summary_results() - 결과 확인")
print("4. get_detailed_summary(team_evaluation_id, period_id) - 상세 요약 보기")

print("\n💡 사용 예시:")
print("process_specific_team(1, 4)  # 1번 팀 평가의 4분기 데이터 처리")
print("get_detailed_summary(1, 4)   # 1번 팀 평가의 4분기 요약 내용 보기")

# 자동 실행을 원한다면 아래 주석 해제
process_all_teams()


🎯 팀별 피드백 요약 에이전트 준비 완료!

실행 옵션:
1. process_all_teams() - 모든 팀 처리
2. process_specific_team(team_evaluation_id, period_id) - 특정 팀 처리
3. check_summary_results() - 결과 확인
4. get_detailed_summary(team_evaluation_id, period_id) - 상세 요약 보기

💡 사용 예시:
process_specific_team(1, 4)  # 1번 팀 평가의 4분기 데이터 처리
get_detailed_summary(1, 4)   # 1번 팀 평가의 4분기 요약 내용 보기
🚀 팀별 피드백 요약 처리 시작
📊 요약 대상 팀: 6개

📋 [1/6] AI개발팀 처리 중...
🤖 AI개발팀: 요약 생성 중... (피드백 4개)
✅ AI개발팀: 완료

📋 [2/6] 기업영업팀 처리 중...
🤖 기업영업팀: 요약 생성 중... (피드백 1개)
✅ 기업영업팀: 완료

📋 [3/6] 데이터분석팀 처리 중...
🤖 데이터분석팀: 요약 생성 중... (피드백 3개)
✅ 데이터분석팀: 완료

📋 [4/6] 백엔드개발팀 처리 중...
🤖 백엔드개발팀: 요약 생성 중... (피드백 2개)
✅ 백엔드개발팀: 완료

📋 [5/6] 퍼포먼스마케팅팀 처리 중...
🤖 퍼포먼스마케팅팀: 요약 생성 중... (피드백 1개)
✅ 퍼포먼스마케팅팀: 완료

📋 [6/6] 프론트엔드팀 처리 중...
🤖 프론트엔드팀: 요약 생성 중... (피드백 2개)
✅ 프론트엔드팀: 완료

🎯 처리 완료: 6/6 성공
📊 성공률: 100.0%
